# MobileNet Fine-Tuning on Multiple Datasets  
### On-Device AI: Efficient CNNs for Real-Time Object Recognition  
**Team Members:** Sanchita, Simran, Leoul, Sandeep  
**Role:** Sandeep (MobileNet Lead)


**Import usual libraries.**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
import os
import time



**2. CIFAR-100 Data Loaders.**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

# --- Transforms ---
# Using ImageNet normalization because we're using an ImageNet-pretrained MobileNet
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

train_tfms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

test_tfms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# --- Datasets & loaders (CIFAR-100) ---
trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=train_tfms)
testset  = datasets.CIFAR100(root='./data', train=False, download=True, transform=test_tfms)

batch_size = 128
num_workers = 2

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True,  num_workers=num_workers)
testloader  = DataLoader(testset,  batch_size=batch_size, shuffle=False, num_workers=num_workers)

classes = trainset.classes  # list of 100 class names
print(f"Train: {len(trainset)}, Test: {len(testset)}, Classes: {len(classes)}")


Device: cuda
Train: 50000, Test: 10000, Classes: 100


**3. Build MobileNet (pretrained) for 100 classes**

In [ ]:

# Load ImageNet-pretrained MobileNetV3-Small
weights = MobileNet_V3_Small_Weights.IMAGENET1K_V1
model = mobilenet_v3_small(weights=weights)

# Replace classifier head for 100 classes
in_feats = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(in_feats, 100)

model = model.to(device)
print(model.classifier)



Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1024, out_features=100, bias=True)
)


**4. Loss & optimizer**

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=5e-4)
# Optional but helpful:
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)


**5. Training loops**

In [ ]:
def train(model, trainloader, epochs=20, print_every=1000):
    model.train()
    for epoch in range(1, epochs + 1):
        running_loss = 0.0
        for i, (images, labels) in enumerate(trainloader, start=1):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % print_every == 0:
                print(f"[Epoch {epoch}, Batch {i}] loss: {running_loss/print_every:.4f}")
                running_loss = 0.0
        if 'scheduler' in globals() and scheduler is not None:
            scheduler.step()
        # epoch summary
        if running_loss > 0:
            print(f"[Epoch {epoch}, End] loss: {running_loss/(len(trainloader)%print_every):.4f}")

print("Starting training…")
train(model, trainloader, epochs=10, print_every=500)  # adjust epochs as needed
print("Finished Training")



Starting training…
[Epoch 1, End] loss: 1.5228
[Epoch 2, End] loss: 0.9427
[Epoch 3, End] loss: 0.8366
[Epoch 4, End] loss: 0.8125
[Epoch 5, End] loss: 0.8080
[Epoch 6, End] loss: 0.8071
[Epoch 7, End] loss: 0.8215
[Epoch 8, End] loss: 0.8506
[Epoch 9, End] loss: 0.8685
[Epoch 10, End] loss: 0.8907
Finished Training


**Evaluation**

In [ ]:
# Test Accurancy
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

accuracy = 100 * correct / total
print(f"\nTest Accuracy on CIFAR-100: {accuracy:.2f}%")



Test Accuracy on CIFAR-100: 46.48%

Test Accuracy on CIFAR-100: 46.48%


**Model Size**

In [ ]:
# Model Size
save_path = "mobilenet_cifar100.pth"
torch.save(model.state_dict(), save_path)

size_mb = os.path.getsize(save_path) / (1024 * 1024)
print(f"Model Size: {size_mb:.2f} MB")


Model Size: 6.31 MB


**Inference Speed (ms/image + FPS)**

In [ ]:
# Inference Speed (ms/image + FPS)

model.eval()
images, _ = next(iter(testloader))
images = images.to(device)

# warmup
for _ in range(10):
    _ = model(images)

# measure
start = time.time()
for _ in range(50):
    _ = model(images)
end = time.time()

avg_batch_time = (end - start) / 50
ms_per_image = (avg_batch_time / images.size(0)) * 1000
fps = images.size(0) / avg_batch_time

print(f"Inference Time Per Image: {ms_per_image:.3f} ms")
print(f"FPS: {fps:.2f}")


Inference Time Per Image: 0.232 ms
FPS: 4303.85


In [ ]:
print("\n===== Performance Summary (MobileNet + CIFAR-100) =====")
print(f"Accuracy:           {accuracy:.2f}%")
print(f"Model Size:         {size_mb:.2f} MB")
print(f"Inference (ms/img): {ms_per_image:.3f} ms")
print(f"FPS:                {fps:.2f}")
print(f"Training Time:      30 min")
print("========================================================")


===== Performance Summary (MobileNet + CIFAR-100) =====
Accuracy:           46.48%
Model Size:         6.31 MB
Inference (ms/img): 0.232 ms
FPS:                4303.85
Training Time:      30 min


### Foods101 Dataset


**Import usual libraries.**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms, models
from torchvision.datasets import Food101
import time
import os

**2. Food-100 Data Loaders.**

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


trainset = Food101(root='./data', split='train', transform=train_transform, download=True)
testset = Food101(root='./data', split='test', transform=test_transform, download=True)


trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)


100%|██████████| 5.00G/5.00G [04:19<00:00, 19.2MB/s]


**Define MobileNetV3-Small for 101 Classes**

In [ ]:
model = models.mobilenet_v3_small(weights=None)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 101)  # 101 classes
model = model.to(device)


**Loss + Optimizer**

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

**Training Function**

In [ ]:
import time

def train(model, trainloader, epochs=10, print_every=500):
    model.train()
    for epoch in range(1, epochs + 1):
        running_loss = 0.0
        for i, (images, labels) in enumerate(trainloader, start=1):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % print_every == 0:
                print(f"[Epoch {epoch}, Batch {i}] loss: {running_loss/print_every:.4f}")
                running_loss = 0.0

    print("Finished Training")

    return True


# ---------- TRAIN + TIME ----------
start_time = time.time()

train(model, trainloader, epochs=10, print_every=500)

end_time = time.time()
total_training_time = (end_time - start_time) / 60
print(f"Training Time: {total_training_time:.2f} min")


[Epoch 1, Batch 500] loss: 4.4002
[Epoch 1, Batch 1000] loss: 4.1209
[Epoch 1, Batch 1500] loss: 3.9350
[Epoch 1, Batch 2000] loss: 3.7895
[Epoch 2, Batch 500] loss: 3.5932
[Epoch 2, Batch 1000] loss: 3.5045
[Epoch 2, Batch 1500] loss: 3.4155
[Epoch 2, Batch 2000] loss: 3.3418
[Epoch 3, Batch 500] loss: 3.1668
[Epoch 3, Batch 1000] loss: 3.1312
[Epoch 3, Batch 1500] loss: 3.0698
[Epoch 3, Batch 2000] loss: 2.9971
[Epoch 4, Batch 500] loss: 2.8515
[Epoch 4, Batch 1000] loss: 2.8173
[Epoch 4, Batch 1500] loss: 2.7640
[Epoch 4, Batch 2000] loss: 2.7386
[Epoch 5, Batch 500] loss: 2.6009
[Epoch 5, Batch 1000] loss: 2.5833
[Epoch 5, Batch 1500] loss: 2.5403
[Epoch 5, Batch 2000] loss: 2.5329
[Epoch 6, Batch 500] loss: 2.4051
[Epoch 6, Batch 1000] loss: 2.3892
[Epoch 6, Batch 1500] loss: 2.3589
[Epoch 6, Batch 2000] loss: 2.3291
[Epoch 7, Batch 500] loss: 2.2211
[Epoch 7, Batch 1000] loss: 2.2072
[Epoch 7, Batch 1500] loss: 2.2032
[Epoch 7, Batch 2000] loss: 2.1928
[Epoch 8, Batch 500] loss: 

**Evaluation**

In [ ]:
# Test Accuracy on Food101
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

accuracy = 100 * correct / total
print(f"\nTest Accuracy on Food101: {accuracy:.2f}%")



Test Accuracy on Food101: 54.34%


**Measure Model Size**

In [ ]:
import os
torch.save(model.state_dict(), "temp_model.pth")
size_mb = os.path.getsize("temp_model.pth") / (1024 * 1024)
os.remove("temp_model.pth")

print(f"Model Size: {size_mb:.2f} MB")


Model Size: 6.31 MB


**Inference Time (ms per image & FPS)**

In [ ]:
import time

def measure_inference_speed(model, device, num_runs=200):
    model.eval()

    data_iter = iter(testloader)
    images, _ = next(data_iter)
    images = images.to(device)

    # warm-up
    for _ in range(20):
        _ = model(images)

    # timing
    start = time.time()
    for _ in range(num_runs):
        _ = model(images)
    end = time.time()

    total_time = end - start
    ms_per_image = (total_time / (num_runs * images.size(0))) * 1000
    fps = 1000 / ms_per_image

    print(f"Inference Time Per Image: {ms_per_image:.3f} ms")
    print(f"FPS: {fps:.2f}")

    return ms_per_image, fps

ms_per_image, fps = measure_inference_speed(model, device)


Inference Time Per Image: 0.330 ms
FPS: 3028.05


**Final Performance Summary**

In [ ]:
print("\n===== Performance Summary (MobileNet + Food101) =====")
print(f"Accuracy:           {accuracy:.2f}%")
print(f"Model Size:         {size_mb:.2f} MB")
print(f"Inference (ms/img): {ms_per_image:.3f} ms")
print(f"FPS:                {fps:.2f}")
print(f"Training Time:      {total_training_time:.2f} min")
print("=====================================================\n")



===== Performance Summary (MobileNet + Food101) =====
Accuracy:           54.34%
Model Size:         6.31 MB
Inference (ms/img): 0.330 ms
FPS:                3028.05
Training Time:      63.18 min



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import time
import os

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # convert 1 channel → 3 channels
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
train_data = datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

test_data = datasets.FashionMNIST(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

100%|██████████| 26.4M/26.4M [00:01<00:00, 13.9MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 205kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.85MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 14.4MB/s]


In [ ]:
trainloader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=4)
testloader = DataLoader(test_data, batch_size=64, shuffle=False, num_workers=4)

print("Train batches:", len(trainloader))
print("Test batches:", len(testloader))

Train batches: 938
Test batches: 157


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

model = models.mobilenet_v3_small(pretrained=True)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 10)
model = model.to(device)



In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
import time

def train_fmnist(model, trainloader, epochs=10, print_every=500):
    model.train()
    running_loss = 0.0

    for epoch in range(1, epochs + 1):
        running_loss = 0.0

        for i, (images, labels) in enumerate(trainloader, start=1):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % print_every == 0:
                print(f"[Epoch {epoch}, Batch {i}] loss: {running_loss/print_every:.4f}")
                running_loss = 0.0

        # epoch end message
        print(f"[Epoch {epoch}, End] loss: {running_loss/len(trainloader):.4f}")

    print("Finished Training")
    return True


# ======== TRAIN + TIME ==========
start_time = time.time()

train_fmnist(model, trainloader, epochs=10, print_every=500)

end_time = time.time()
total_training_time = (end_time - start_time) / 60

print(f"\nTraining Time: {total_training_time:.2f} min")


[Epoch 1, Batch 500] loss: 0.3169
[Epoch 1, End] loss: 0.1025
[Epoch 2, Batch 500] loss: 0.1806
[Epoch 2, End] loss: 0.0859
[Epoch 3, Batch 500] loss: 0.1530
[Epoch 3, End] loss: 0.0723
[Epoch 4, Batch 500] loss: 0.1316
[Epoch 4, End] loss: 0.0642
[Epoch 5, Batch 500] loss: 0.1188
[Epoch 5, End] loss: 0.0578
[Epoch 6, Batch 500] loss: 0.1011
[Epoch 6, End] loss: 0.0529
[Epoch 7, Batch 500] loss: 0.0874
[Epoch 7, End] loss: 0.0445
[Epoch 8, Batch 500] loss: 0.0752
[Epoch 8, End] loss: 0.0394
[Epoch 9, Batch 500] loss: 0.0629
[Epoch 9, End] loss: 0.0367
[Epoch 10, Batch 500] loss: 0.0500
[Epoch 10, End] loss: 0.0296
Finished Training

Training Time: 19.43 min


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

accuracy_fmnist = 100 * correct / total
print(f"\nTest Accuracy on Fashion-MNIST: {accuracy_fmnist:.2f}%")


Test Accuracy on Fashion-MNIST: 93.72%


In [ ]:
import os
torch.save(model.state_dict(), "mobilenet_fmnist.pth")

size_mb_fmnist = os.path.getsize("mobilenet_fmnist.pth") / (1024 * 1024)
print(f"Model Size: {size_mb_fmnist:.2f} MB")

Model Size: 5.96 MB


In [ ]:
import time

model.eval()
num_images = 200
images_seen = 0
total_time = 0.0

with torch.no_grad():
    for images, _ in testloader:
        images = images.to(device)

        start = time.time()
        _ = model(images)
        end = time.time()

        batch_time = end - start
        total_time += batch_time
        images_seen += images.size(0)

        if images_seen >= num_images:
            break

ms_per_image_fmnist = (total_time / images_seen) * 1000
fps_fmnist = 1 / (ms_per_image_fmnist / 1000)

print(f"\nInference Time Per Image: {ms_per_image_fmnist:.3f} ms")
print(f"FPS: {fps_fmnist:.2f}")



Inference Time Per Image: 0.439 ms
FPS: 2280.09


In [ ]:
print("\n===== Performance Summary (MobileNet + Fashion-MNIST) =====")
print(f"Accuracy:            {accuracy_fmnist:.2f}%")
print(f"Model Size:          {size_mb_fmnist:.2f} MB")
print(f"Inference (ms/img):  {ms_per_image_fmnist:.3f} ms")
print(f"FPS:                 {fps_fmnist:.2f}")
print(f"Training Time:       {total_training_time:.2f} min")
print("============================================================")



===== Performance Summary (MobileNet + Fashion-MNIST) =====
Accuracy:            93.72%
Model Size:          5.96 MB
Inference (ms/img):  0.439 ms
FPS:                 2280.09
Training Time:       19.43 min


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models
import time
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [ ]:
# ======== DATA TRANSFORMS FOR STANFORD CARS =========
from torchvision.datasets import QMNIST

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor()
])

trainset = QMNIST(root="./data", train=True, download=True, transform=transform)
testset  = QMNIST(root="./data", train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader  = DataLoader(testset, batch_size=64, shuffle=False)

print("Train batches:", len(trainloader))
print("Test batches:", len(testloader))

100%|██████████| 9.70M/9.70M [00:00<00:00, 457MB/s]
100%|██████████| 463k/463k [00:00<00:00, 69.9MB/s]
100%|██████████| 9.74M/9.74M [00:00<00:00, 461MB/s]
100%|██████████| 527k/527k [00:00<00:00, 124MB/s]

Train batches: 938
Test batches: 938


In [ ]:
import torch
import torch.nn as nn
from torchvision.models import mobilenet_v3_small

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = mobilenet_v3_small(weights="IMAGENET1K_V1")
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 10)  # QMNIST has 10 classes

model = model.to(device)


Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


100%|██████████| 9.83M/9.83M [00:00<00:00, 160MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
import time

def train_qmnist(model, trainloader, epochs=10, print_every=500):
    model.train()
    running_loss = 0.0

    for epoch in range(1, epochs + 1):
        running_loss = 0.0

        for i, (images, labels) in enumerate(trainloader, start=1):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % print_every == 0:
                print(f"[Epoch {epoch}, Batch {i}] loss: {running_loss/print_every:.4f}")
                running_loss = 0.0

        # epoch end message
        print(f"[Epoch {epoch}, End] loss: {running_loss/len(trainloader):.4f}")

    print("Finished Training")
    return True


# ========= TRAIN + TIME =========
start_time = time.time()

train_qmnist(model, trainloader, epochs=10, print_every=500)

end_time = time.time()
total_training_time = (end_time - start_time) / 60

print(f"\nTraining Time: {total_training_time:.2f} min")


[Epoch 1, Batch 500] loss: 0.0920
[Epoch 1, End] loss: 0.0200
[Epoch 2, Batch 500] loss: 0.0311
[Epoch 2, End] loss: 0.0142
[Epoch 3, Batch 500] loss: 0.0207
[Epoch 3, End] loss: 0.0124
[Epoch 4, Batch 500] loss: 0.0208
[Epoch 4, End] loss: 0.0121
[Epoch 5, Batch 500] loss: 0.0181
[Epoch 5, End] loss: 0.0098
[Epoch 6, Batch 500] loss: 0.0188
[Epoch 6, End] loss: 0.0094
[Epoch 7, Batch 500] loss: 0.0136
[Epoch 7, End] loss: 0.0091
[Epoch 8, Batch 500] loss: 0.0151
[Epoch 8, End] loss: 0.0088
[Epoch 9, Batch 500] loss: 0.0141
[Epoch 9, End] loss: 0.0073
[Epoch 10, Batch 500] loss: 0.0098
[Epoch 10, End] loss: 0.0068
Finished Training

Training Time: 23.39 min


In [ ]:
# ===== Test Accuracy (QMNIST) =====
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

accuracy = 100 * correct / total
print(f"\nTest Accuracy on QMNIST: {accuracy:.2f}%")



Test Accuracy on QMNIST: 99.10%


In [ ]:
# ===== Model Size (QMNIST) =====
torch.save(model.state_dict(), "temp_qmnist_model.pth")
import os

size_mb = os.path.getsize("temp_qmnist_model.pth") / (1024 * 1024)
print(f"\nModel Size: {size_mb:.2f} MB")



Model Size: 5.96 MB


In [ ]:
# ===== Inference Time per Image (QMNIST) =====
import time

model.eval()
dummy_input = torch.randn(1, 3, 224, 224).to(device)  # QMNIST is grayscale (1 channel)

# warmup
for _ in range(10):
    _ = model(dummy_input)

# timing
start = time.time()
runs = 200
for _ in range(runs):
    _ = model(dummy_input)
end = time.time()

avg_ms = ((end - start) / runs) * 1000
fps = 1000 / avg_ms

print(f"\nInference Time Per Image: {avg_ms:.3f} ms")
print(f"FPS: {fps:.2f}")



Inference Time Per Image: 6.282 ms
FPS: 159.18


In [ ]:
print("\n===== Performance Summary (MobileNet + QMNIST) =====")
print(f"Accuracy:           {accuracy:.2f}%")
print(f"Model Size:         {size_mb:.2f} MB")
print(f"Inference (ms/img): {avg_ms:.3f} ms")
print(f"FPS:                {fps:.2f}")
print("----------------------------------------------------")



===== Performance Summary (MobileNet + QMNIST) =====
Accuracy:           99.10%
Model Size:         5.96 MB
Inference (ms/img): 6.282 ms
FPS:                159.18
----------------------------------------------------
